In [ ]:
import botocore
import boto3
import threading, logging, time
from kafka import KafkaProducer
import smart_open

class Producer(threading.Thread):
    daemon = True

    def run(self):
        producer = KafkaProducer(bootstrap_servers='ec2-52-9-120-149.us-west-1.compute.amazonaws.com:9092')

        bucket_name = 'moyang28'
        bucket = self.__get_s3_bucket__(bucket_name)
        # Send data from S3 to Kafka queue
        
        print("before")
        for i in range(1,2):
            for line in  smart_open.smart_open("s3://moyang28/trainSearchStream.tsv"):
#                 print(line)
                producer.send('topic_click', line)
        print("done")
                
    # Access S3 bucket
    def __get_s3_bucket__(self, bucket_name):
        s3 = boto3.resource('s3')
        try:
            s3.meta.client.head_bucket(Bucket=bucket_name)
        except botocore.exceptions.ClientError as e:
            # If a client error is thrown, check that it was a 404 error.
            # If it was a 404 error, then the bucket does not exist.
            error_code = int(e.response['Error']['Code'])
            if error_code == 404:
                exists = False
                print (e.response['404 Error: bucket not found'])
            else:
                print (e.response['Error'])
            return None
        else:
            return s3.Bucket(bucket_name)


if __name__ == "__main__":
#     Producer().run()
    Producer().start()
    while True:
        time.sleep(10)


before
